# Operar con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas entre elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, exponenciales y logarítmicas, etc.).
Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs que introdujimos en [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) son la clave.

Sin embargo, Pandas incluye un par de giros útiles: para las operaciones unarias como la negación y las funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para las operaciones binarias como la suma y la multiplicación, Pandas *alineará automáticamente los índices* al pasar los objetos a la ufunc.
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes -ambas tareas potencialmente propensas a errores con arrays de NumPy sin procesar- se convierten en tareas esencialmente infalibles con Pandas.
Además, veremos que hay operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Preservación del índice

Como Pandas está diseñado para trabajar con NumPy, cualquier ufunc de NumPy funcionará con los objetos ``Series`` y ``DataFrame`` de Pandas.
Empecemos definiendo una simple ``Serie`` y ``DataFrame`` en la que demostrar esto:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Si aplicamos una ufunc NumPy sobre cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

In [ ]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:

In [ ]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Cualquiera de las ufuncs discutidas en [Computación en Arrays NumPy: Funciones Universales](02.03-Computación-en-arrays-ufuncs.ipynb) puede ser usada de manera similar.

## UFuncs: Alineación de índices

Para operaciones binarias sobre dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realizar la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen.

### Alineación del índice en la serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes, y encontramos sólo los tres primeros estados de EE.UU. por *área* y los tres primeros estados de EE.UU. por *población*:

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

Veamos qué ocurre cuando los dividimos para calcular la densidad de población:

In [ ]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

El array resultante contiene la *unión* de los índices de los dos arrays de entrada, que podría determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:

In [ ]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el que uno u otro no tenga una entrada se marca con ``NaN``, o "Not a Number", que es como Pandas marca los datos que faltan (ver más discusión sobre los datos que faltan en [Handling Missing Data](03.04-Missing-Values.ipynb)).
Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas incorporadas de Python; cualquier valor que falte se rellena con NaN por defecto:

In [ ]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si el uso de valores NaN no es el comportamiento deseado, el valor de llenado puede ser modificado usando métodos de objetos apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento de ``A`` o ``B`` que pueda faltar:

In [ ]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Alineación de índices en DataFrame

Un tipo de alineación similar tiene lugar para *tanto* las columnas como los índices cuando se realizan operaciones en ``DataFrame``:

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [ ]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


Observe que los índices se alinean correctamente independientemente de su orden en los dos objetos, y los índices en el resultado están ordenados.
Al igual que en el caso de ``Series``, podemos utilizar el método aritmético del objeto asociado y pasarle cualquier ``valor de relleno`` que queramos utilizar en lugar de las entradas que falten.
Aquí rellenaremos con la media de todos los valores de ``A`` (calculada apilando primero las filas de ``A``):

In [ ]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


La siguiente tabla enumera los operadores de Python y sus métodos equivalentes de los objetos de Pandas:

| Operador de Python | Método(s) de Pandas                 |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Cuando se realizan operaciones entre un ``DataFrame`` y una ``Serie``, el índice y la alineación de las columnas se mantienen de forma similar.
Las operaciones entre un ``DataFrame`` y una ``Serie`` son similares a las operaciones entre un array bidimensional y uno unidimensional de NumPy.
Consideremos una operación común, donde encontramos la diferencia de un array bidimensional y una de sus filas:

In [ ]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [ ]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

De acuerdo con las reglas de difusión de NumPy (ver [Computación en arrays: difusión](02.05-Computación-en-arrays-difusión.ipynb)), la sustracción entre un array bidimensional y una de sus filas se aplica por filas.

En Pandas, la convención opera de forma similar en las filas por defecto:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


Si, en cambio, desea operar por columnas, puede utilizar los métodos del objeto mencionados anteriormente, especificando la palabra clave ``eje``:

In [ ]:
df.subtract(df['R'], axis=0)

Tenga en cuenta que estas operaciones ``DataFrame``/`Series``, al igual que las operaciones comentadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [ ]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [ ]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


Esta preservación y alineación de índices y columnas significa que las operaciones sobre los datos en Pandas siempre mantendrán el contexto de los datos, lo que evita los tipos de errores tontos que podrían surgir al trabajar con datos heterogéneos y/o mal alineados en arrays NumPy en bruto.